<a href="https://colab.research.google.com/github/teyang-lau/you-only-edit-once/blob/main/scripts/extract_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install library requirements to read videos from YouTube. youtube-dl==2020.12.2 allows bypassing the 'dislike counts' bug which causes the code to terminate.

Source: https://stackoverflow.com/questions/70297028/i-have-a-problem-with-dislike-error-when-creating-an-pafy-new-url-perhaps-its

In [1]:
!pip install -q pafy
!pip install -q youtube-dl==2020.12.2

Import libraries

In [1]:
import os
import uuid
import cv2
# import pafy
# import youtube_dl

Extract frames from video

In [2]:
def video2frames(video_file, output_path, factor=1, youtube=False):

    """Extract frames from a video file or youtube link
    
    Args:
    video_file (str): path to the video
    output_path (str): path to output folder for storing extracted frames
    factor (int): how many seconds to extract 1 frame. 1 = extract a frame every sec, 2 = extract a frame every 2 secs
    youtube (bool): whether to get video directly from youtube link

    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    if youtube == False:
        vid = cv2.VideoCapture(video_file)
    
    elif youtube == True:
        video = pafy.new(video_file)
        best = video.getbest(preftype="mp4")
        vid = cv2.VideoCapture(best.url)
    
    fps = round(vid.get(cv2.CAP_PROP_FPS))
    num_frames = int(vid. get(cv2. CAP_PROP_FRAME_COUNT))
    index = 0        
    while vid.isOpened():
        success, img = vid.read()
        index += 1
        if success:
            # extract every fps frame of the video, multplied by a factor
            # factor of 1 = extract a frame every sec, 2 = extract a frame every 2 secs
            if index % (fps*factor) == 0:
                cv2.imwrite(output_path + '/' + str(uuid.uuid4()) + '_' + str(index) + '.jpg', img)
        # stop reading at end of video
        # need this as some frames return False success, so cannot
        # use success to break the while loop
        if index > num_frames:
            break
    vid.release()
    
    return 

Extracting frames from youtube video

In [ ]:
url = "https://www.youtube.com/watch?v=ptygf0cPtUk"
video2frames(video_file = url,
             output_path = './data/extracted_frames_2',
             factor = 1,
             youtube=True)

In [49]:
from google.colab import files
files.download("/content/extracted_frames.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
video2frames( '../data/videos/GOPR0247.MP4', '../data/extracted_frames/', factor=3 )

Function to extract frames from multiple videos

In [ ]:
def multiple_video2frames( video_path, output_path , ignore_prev_vid=False, prev_vid_file=None):

    """Extract frames from multple videos file
    
    Args:
    video_path (str): path to folder containing all videos
    output_path (str): path to output folder for storing extracted frames
    ignore_prev_vid (bool): whether to ignore previous vidoes that have been already extracted
    prev_vid_file (str): path to text file containing previously extracted video filenames

    """

    vid_count = 0

    if ignore_prev_vid:
        file = open(prev_vid_file)
        text = file.readlines()
        prev_vids = {t.rstrip('\n') : True for t in text} 
        file.close()
        file = open(prev_vid_file, 'a+')

    list_videos = os.listdir(video_path)
    print('Found {} videos'.format(len(list_videos)))
    for video in list_videos:
        # skip video if extracted before
        if ignore_prev_vid and video in prev_vids:
            continue
        # read and extract frame
        vid_count += 1
        print('Extracting Video {}'.format(vid_count))
        video_file = video_path + '/' + video
        video2frames(video_file, output_path)
        # add video name to ignore_prev_vid file
        if ignore_prev_vid:
            file.write(video + '\n')

    if vid_count > 0: 
        print('Extraction Completed!')
            
    return

In [ ]:
multiple_video2frames('../data/videos', '../data/extracted_frames', '../data/ignore_prev_vid.txt', True)